In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup
import re
from pathlib import Path
import hashlib

# Configuration
OLLAMA_BASE_URL = "http://localhost:11434"
OLLAMA_MODEL = "llama3.2" # or whichever llama 3.2 model you have pulled in Ollama

# Create a simple vector store for document chunks
class SimpleVectorStore:
    def __init__(self):
        self.documents = []
        self.embeddings = []
        
    def add_document(self, content, metadata=None):
        doc_id = len(self.documents)
        self.documents.append({
            "id": doc_id,
            "content": content,
            "metadata": metadata or {}
        })
        # Get embedding from Ollama
        embedding = self._get_embedding(content)
        self.embeddings.append(embedding)
        return doc_id
    
    def _get_embedding(self, text):
        """Get embedding from Ollama"""
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/embeddings",
            json={"model": OLLAMA_MODEL, "prompt": text}
        )
        if response.status_code == 200:
            return response.json().get("embedding", [])
        else:
            print(f"Error getting embedding: {response.text}")
            # Return empty embedding as fallback
            return []
    
    def _similarity(self, embedding1, embedding2):
        """Calculate cosine similarity between two embeddings"""
        if not embedding1 or not embedding2:
            return 0
        
        dot_product = sum(a * b for a, b in zip(embedding1, embedding2))
        magnitude1 = sum(a * a for a in embedding1) ** 0.5
        magnitude2 = sum(b * b for b in embedding2) ** 0.5
        
        if magnitude1 * magnitude2 == 0:
            return 0
            
        return dot_product / (magnitude1 * magnitude2)
    
    def search(self, query, top_k=3):
        """Search for similar documents"""
        query_embedding = self._get_embedding(query)
        
        similarities = []
        for i, doc_embedding in enumerate(self.embeddings):
            similarity = self._similarity(query_embedding, doc_embedding)
            similarities.append((i, similarity))
        
        # Sort by similarity (descending)
        similarities.sort(key=lambda x: x[1], reverse=True)
        
        # Return top k documents
        results = []
        for i, similarity in similarities[:top_k]:
            doc = self.documents[i].copy()
            doc["similarity"] = similarity
            results.append(doc)
            
        return results

# HTML Parser to extract relevant information
def parse_html_to_text(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract relevant components
    components = []
    
    # Extract structure and IDs
    for tag in soup.find_all(True):
        if tag.name in ['div', 'button', 'input', 'ul', 'li'] and tag.get('id'):
            components.append(f"{tag.name}#{tag.get('id')}: {tag.get('class', '')}")
        elif tag.name in ['button', 'input'] and tag.get('class'):
            components.append(f"{tag.name}.{' '.join(tag.get('class', []))}: {tag.get('placeholder', '')}")
    
    # Extract JavaScript functionality
    scripts = soup.find_all('script')
    js_functionality = []
    for script in scripts:
        if script.string:
            # Extract function names
            functions = re.findall(r'function\s+(\w+)', script.string)
            event_listeners = re.findall(r'addEventListener\([\'\"](\w+)[\'\"]', script.string)
            variables = re.findall(r'const\s+(\w+)', script.string)
            
            if functions:
                js_functionality.append(f"Functions: {', '.join(functions)}")
            if event_listeners:
                js_functionality.append(f"Event Listeners: {', '.join(event_listeners)}")
            if variables:
                js_functionality.append(f"Key Variables: {', '.join(variables)}")
    
    return "\n".join(components + js_functionality)

# Load and process the todo app HTML
def process_todo_app(html_content):
    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract title and main elements
    title = soup.title.string if soup.title else "Todo App"
    
    # Extract the main structure
    structure = parse_html_to_text(html_content)
    
    # Extract styling cues
    styles = []
    style_tags = soup.find_all('style')
    for style in style_tags:
        if style.string:
            class_matches = re.findall(r'\.(\w+)\s*{', style.string)
            id_matches = re.findall(r'#(\w+)\s*{', style.string)
            if class_matches:
                styles.append(f"CSS Classes: {', '.join(class_matches)}")
            if id_matches:
                styles.append(f"CSS IDs: {', '.join(id_matches)}")
    
    # Extract JavaScript functions
    script_tags = soup.find_all('script')
    functions = []
    for script in script_tags:
        if script.string:
            fn_matches = re.findall(r'function\s+(\w+)', script.string)
            if fn_matches:
                functions.append(f"JS Functions: {', '.join(fn_matches)}")
    
    # Create chunks for the vector store
    chunks = [
        {
            "content": f"Todo App Overview: {title}\n\nMain Components:\n{structure}",
            "metadata": {"type": "structure"}
        },
        {
            "content": f"Todo App Styling:\n{chr(10).join(styles)}",
            "metadata": {"type": "styling"}
        },
        {
            "content": f"Todo App Functionality:\n{chr(10).join(functions)}",
            "metadata": {"type": "functionality"}
        }
    ]
    
    # Extract specific functionality from JavaScript
    for script in script_tags:
        if script.string:
            # Look for specific functions
            add_todo_match = re.search(r'function\s+addTodo\(\)\s*{([^}]+)}', script.string, re.DOTALL)
            if add_todo_match:
                chunks.append({
                    "content": f"Add Todo Functionality:\n{add_todo_match.group(1).strip()}",
                    "metadata": {"type": "function", "name": "addTodo"}
                })
            
            delete_todo_match = re.search(r'function\s+deleteTodo\([^)]+\)\s*{([^}]+)}', script.string, re.DOTALL)
            if delete_todo_match:
                chunks.append({
                    "content": f"Delete Todo Functionality:\n{delete_todo_match.group(1).strip()}",
                    "metadata": {"type": "function", "name": "deleteTodo"}
                })
            
            toggle_todo_match = re.search(r'function\s+toggleTodo\([^)]+\)\s*{([^}]+)}', script.string, re.DOTALL)
            if toggle_todo_match:
                chunks.append({
                    "content": f"Toggle Todo Functionality:\n{toggle_todo_match.group(1).strip()}",
                    "metadata": {"type": "function", "name": "toggleTodo"}
                })
            
            filter_match = re.search(r'filterButtons.forEach\(button\s*=>\s*{([^}]+)}', script.string, re.DOTALL)
            if filter_match:
                chunks.append({
                    "content": f"Filter Functionality:\n{filter_match.group(1).strip()}",
                    "metadata": {"type": "function", "name": "filter"}
                })
    
    return chunks

# Function to generate a Playwright test using Ollama
def generate_playwright_test(query, context_text):
    prompt = f"""You are a test automation expert. Write a Playwright test for a Todo List application based on the following information and query.

APPLICATION CONTEXT:
{context_text}

QUERY:
{query}

TASK:
Write a complete, runnable Playwright test in TypeScript that tests the functionality mentioned in the query.
Use page objects pattern where appropriate.
Include imports, setup, and teardown.
Add detailed comments explaining the test steps.
Make sure the selectors match the HTML elements in the context.
Make the test robust with appropriate waits and assertions.

FORMAT:
```typescript
// Todo App Test - [Test Name]
// Imports and setup code...
// Page object class if needed...
// Test code...
```

"""
    try:
        response = requests.post(
            f"{OLLAMA_BASE_URL}/api/generate",
            json={"model": OLLAMA_MODEL, "prompt": prompt, "stream": False}
        )
        response.raise_for_status()
        result = response.json()
        return result.get("response", "Error: No response generated")
    except Exception as e:
        return f"Error generating test: {str(e)}"

# Cache for storing generated tests
class TestCache:
    def __init__(self, cache_dir="test_cache"):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
    
    def get_cache_key(self, query, context):
        """Generate a cache key from query and context"""
        combined = query + context
        return hashlib.md5(combined.encode()).hexdigest()
    
    def get_from_cache(self, query, context):
        """Retrieve from cache if exists"""
        cache_key = self.get_cache_key(query, context)
        cache_file = self.cache_dir / f"{cache_key}.txt"
        
        if cache_file.exists():
            return cache_file.read_text()
        return None
    
    def save_to_cache(self, query, context, test_code):
        """Save to cache"""
        cache_key = self.get_cache_key(query, context)
        cache_file = self.cache_dir / f"{cache_key}.txt"
        cache_file.write_text(test_code)

# Main Function to process the todo app and generate tests
def generate_todo_app_tests(html_content, test_queries):
    # Initialize vector store and test cache
    vector_store = SimpleVectorStore()
    test_cache = TestCache()
    
    # Process the HTML and add chunks to vector store
    chunks = process_todo_app(html_content)
    for chunk in chunks:
        vector_store.add_document(chunk["content"], chunk["metadata"])
    
    # Generate tests for each query
    generated_tests = []
    
    for query in test_queries:
        print(f"Generating test for: {query}")
        
        # Search for relevant context
        relevant_docs = vector_store.search(query, top_k=3)
        context_text = "\n\n".join([doc["content"] for doc in relevant_docs])
        
        # Check cache first
        cached_test = test_cache.get_from_cache(query, context_text)
        if cached_test:
            print(f"Found test in cache for: {query}")
            test_code = cached_test
        else:
            # Generate test using Ollama
            test_code = generate_playwright_test(query, context_text)
            # Save to cache
            test_cache.save_to_cache(query, context_text, test_code)
        
        generated_tests.append({
            "query": query,
            "test_code": test_code
        })
    
    return generated_tests

# Execution example
if __name__ == "__main__":
    # Read the HTML content from a file
    todo_app_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Todo List App</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
        }
        
        body {
            background-color: #f5f5f5;
            color: #333;
            min-height: 100vh;
            padding: 20px;
        }
        
        .container {
            max-width: 600px;
            margin: 0 auto;
            background-color: white;
            border-radius: 10px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            padding: 20px;
        }
        
        h1 {
            text-align: center;
            margin-bottom: 20px;
            color: #4285f4;
        }
        
        .input-container {
            display: flex;
            margin-bottom: 20px;
        }
        
        #todo-input {
            flex-grow: 1;
            padding: 10px;
            font-size: 16px;
            border: 1px solid #ddd;
            border-radius: 4px 0 0 4px;
            outline: none;
        }
        
        #add-button {
            background-color: #4285f4;
            color: white;
            border: none;
            padding: 10px 15px;
            cursor: pointer;
            font-size: 16px;
            border-radius: 0 4px 4px 0;
            transition: background-color 0.3s;
        }
        
        #add-button:hover {
            background-color: #3367d6;
        }
        
        .filters {
            display: flex;
            justify-content: center;
            margin-bottom: 20px;
            gap: 10px;
        }
        
        .filter-btn {
            background-color: #f1f1f1;
            border: none;
            padding: 8px 12px;
            cursor: pointer;
            border-radius: 4px;
            font-size: 14px;
            transition: background-color 0.3s;
        }
        
        .filter-btn.active {
            background-color: #4285f4;
            color: white;
        }
        
        .filter-btn:hover {
            background-color: #e4e4e4;
        }
        
        .filter-btn.active:hover {
            background-color: #3367d6;
        }
        
        .todo-list {
            list-style-type: none;
        }
        
        .todo-item {
            display: flex;
            align-items: center;
            padding: 10px;
            border-bottom: 1px solid #eee;
            animation: fadeIn 0.5s;
        }
        
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(-10px); }
            to { opacity: 1; transform: translateY(0); }
        }
        
        .todo-item:last-child {
            border-bottom: none;
        }
        
        .todo-item input[type="checkbox"] {
            margin-right: 10px;
            width: 18px;
            height: 18px;
            cursor: pointer;
        }
        
        .todo-text {
            flex-grow: 1;
            font-size: 16px;
            transition: color 0.3s;
        }
        
        .completed .todo-text {
            text-decoration: line-through;
            color: #888;
        }
        
        .delete-btn {
            background-color: #ff5252;
            color: white;
            border: none;
            border-radius: 4px;
            padding: 5px 10px;
            cursor: pointer;
            margin-left: 10px;
            transition: background-color 0.3s;
        }
        
        .delete-btn:hover {
            background-color: #ff1a1a;
        }
        
        .clear-completed {
            background-color: #f1f1f1;
            border: none;
            padding: 8px 12px;
            cursor: pointer;
            border-radius: 4px;
            font-size: 14px;
            margin-top: 20px;
            transition: background-color 0.3s;
        }
        
        .clear-completed:hover {
            background-color: #e4e4e4;
        }
        
        .empty-message {
            text-align: center;
            margin: 20px 0;
            color: #888;
            font-style: italic;
        }
        
        .stats {
            margin-top: 20px;
            text-align: center;
            color: #888;
            font-size: 14px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Todo List</h1>
        
        <div class="input-container">
            <input type="text" id="todo-input" placeholder="Add a new task...">
            <button id="add-button">Add</button>
        </div>
        
        <div class="filters">
            <button class="filter-btn active" data-filter="all">All</button>
            <button class="filter-btn" data-filter="active">Active</button>
            <button class="filter-btn" data-filter="completed">Completed</button>
        </div>
        
        <ul class="todo-list" id="todo-list"></ul>
        
        <div class="stats" id="stats"></div>
        
        <button class="clear-completed" id="clear-completed">Clear Completed</button>
    </div>

    <script>
        document.addEventListener('DOMContentLoaded', function() {
            // DOM elements
            const todoInput = document.getElementById('todo-input');
            const addButton = document.getElementById('add-button');
            const todoList = document.getElementById('todo-list');
            const filterButtons = document.querySelectorAll('.filter-btn');
            const clearCompletedBtn = document.getElementById('clear-completed');
            const statsElement = document.getElementById('stats');
            
            // App state
            let todos = JSON.parse(localStorage.getItem('todos')) || [];
            let currentFilter = 'all';
            
            // Initialize app
            renderTodos();
            updateStats();
            
            // Event listeners
            addButton.addEventListener('click', addTodo);
            todoInput.addEventListener('keypress', function(e) {
                if (e.key === 'Enter') {
                    addTodo();
                }
            });
            
            clearCompletedBtn.addEventListener('click', clearCompleted);
            
            filterButtons.forEach(button => {
                button.addEventListener('click', () => {
                    // Update active class
                    filterButtons.forEach(btn => btn.classList.remove('active'));
                    button.classList.add('active');
                    
                    // Update filter
                    currentFilter = button.getAttribute('data-filter');
                    renderTodos();
                });
            });
            
            // Functions
            function addTodo() {
                const todoText = todoInput.value.trim();
                
                if (todoText !== '') {
                    const newTodo = {
                        id: Date.now(),
                        text: todoText,
                        completed: false
                    };
                    
                    todos.push(newTodo);
                    saveTodos();
                    renderTodos();
                    updateStats();
                    
                    // Clear input
                    todoInput.value = '';
                    todoInput.focus();
                }
            }
            
            function deleteTodo(id) {
                todos = todos.filter(todo => todo.id !== id);
                saveTodos();
                renderTodos();
                updateStats();
            }
            
            function toggleTodo(id) {
                todos = todos.map(todo => {
                    if (todo.id === id) {
                        return { ...todo, completed: !todo.completed };
                    }
                    return todo;
                });
                
                saveTodos();
                renderTodos();
                updateStats();
            }
            
            function clearCompleted() {
                todos = todos.filter(todo => !todo.completed);
                saveTodos();
                renderTodos();
                updateStats();
            }
            
            function renderTodos() {
                // Clear the list
                todoList.innerHTML = '';
                
                // Filter todos based on current filter
                let filteredTodos = todos;
                if (currentFilter === 'active') {
                    filteredTodos = todos.filter(todo => !todo.completed);
                } else if (currentFilter === 'completed') {
                    filteredTodos = todos.filter(todo => todo.completed);
                }
                
                // Show message if no todos
                if (filteredTodos.length === 0) {
                    const emptyMessage = document.createElement('p');
                    emptyMessage.className = 'empty-message';
                    emptyMessage.textContent = 'No tasks found';
                    todoList.appendChild(emptyMessage);
                    return;
                }
                
                // Render todos
                filteredTodos.forEach(todo => {
                    const todoItem = document.createElement('li');
                    todoItem.className = 'todo-item';
                    if (todo.completed) {
                        todoItem.classList.add('completed');
                    }
                    
                    const checkbox = document.createElement('input');
                    checkbox.type = 'checkbox';
                    checkbox.checked = todo.completed;
                    checkbox.addEventListener('change', () => toggleTodo(todo.id));
                    
                    const todoText = document.createElement('span');
                    todoText.className = 'todo-text';
                    todoText.textContent = todo.text;
                    
                    const deleteBtn = document.createElement('button');
                    deleteBtn.className = 'delete-btn';
                    deleteBtn.textContent = 'Delete';
                    deleteBtn.addEventListener('click', () => deleteTodo(todo.id));
                    
                    todoItem.appendChild(checkbox);
                    todoItem.appendChild(todoText);
                    todoItem.appendChild(deleteBtn);
                    
                    todoList.appendChild(todoItem);
                });
            }
            
            function updateStats() {
                const total = todos.length;
                const completed = todos.filter(todo => todo.completed).length;
                const active = total - completed;
                
                statsElement.textContent = `${total} total • ${active} active • ${completed} completed`;
            }
            
            function saveTodos() {
                localStorage.setItem('todos', JSON.stringify(todos));
            }
        });
    </script>
</body>
</html>
    """
    
    test_queries = [
        "Test adding a new todo item",
        "Test marking a todo as completed",
        "Test deleting a todo item",
        "Test filtering todo items (all, active, completed)",
        "Test clearing completed todos",
        "Test empty input validation",
        "Test todo item persistence after page refresh",
        "End-to-end test of the complete todo app workflow"
    ]
    
    # Generate tests
    generated_tests = generate_todo_app_tests(todo_app_html, test_queries)
    
    # Save generated tests to files
    output_dir = Path("generated_tests")
    output_dir.mkdir(exist_ok=True)
    
    for i, test in enumerate(generated_tests):
        test_file = output_dir / f"test_{i+1}_{test['query'].replace(' ', '_').lower()}.ts"
        
        # Extract actual TypeScript code from the response
        typescript_code = test["test_code"]
        # Clean up any potential markdown code blocks
        typescript_code = re.sub(r'```typescript\s*', '', typescript_code)
        typescript_code = re.sub(r'```\s*$', '', typescript_code)
        
        test_file.write_text(typescript_code)
        print(f"Generated test saved to: {test_file}")

Generating test for: Test adding a new todo item
Generating test for: Test marking a todo as completed
Generating test for: Test deleting a todo item
Generating test for: Test filtering todo items (all, active, completed)
Generating test for: Test clearing completed todos
Generating test for: Test empty input validation
Generating test for: Test todo item persistence after page refresh
Generating test for: End-to-end test of the complete todo app workflow
Generated test saved to: generated_tests\test_1_test_adding_a_new_todo_item.ts
Generated test saved to: generated_tests\test_2_test_marking_a_todo_as_completed.ts
Generated test saved to: generated_tests\test_3_test_deleting_a_todo_item.ts
Generated test saved to: generated_tests\test_4_test_filtering_todo_items_(all,_active,_completed).ts
Generated test saved to: generated_tests\test_5_test_clearing_completed_todos.ts
Generated test saved to: generated_tests\test_6_test_empty_input_validation.ts
Generated test saved to: generated_tes